In [3]:
from transform import preprocessing

df, scaled_df = preprocessing(18, 'standard')
print(df.head())
print(scaled_df.head())

Data has been written to data/preprocessed/18-round-preprocessed.csv
Scaled data has been written to data/preprocessed/18-round-scaled.csv
Data has been written to data/preprocessed/18-round-preprocessed.csv
Scaled data has been written to data/preprocessed/18-round-scaled.csv
   index   선수명    구단 포지션   등번호  출전시간(분)        득점        도움        슈팅  \
0    0.0  가브리엘    강원  FW  10.0    284.0  0.000000  0.000000  1.901408   
1    1.0  가브리엘    광주  MF  11.0   1403.0  0.256593  0.192445  1.539558   
2    2.0  가브리엘    성남  MF  10.0    412.0  0.000000  0.218447  1.747573   
3    3.0   갈레고    강원  MF  11.0    151.0  0.000000  0.000000  4.172185   
4    4.0   강민규  충남아산  MF   9.0   1370.0  0.394161  0.131387  2.890511   

      유효 슈팅  ...        차단        획득        블락       볼미스        파울       피파울  \
0  0.633803  ...  0.000000  2.218310  0.316901  1.901408  0.633803  1.901408   
1  0.641483  ...  1.539558  4.426230  1.667855  1.411262  0.898076  2.116892   
2  0.873786  ...  1.310680  7.645631  0.000

In [4]:
print(df['구단'].value_counts())

구단
김천      38
대전      37
광주      37
전북      35
천안      35
서울      34
포항      34
전남      33
울산      33
인천      33
수원      33
제주      32
충남아산    32
강원      31
서울E     31
성남      31
수원FC    31
안양      31
대구      31
경남      31
김포      30
부천      30
충북청주    29
안산      28
부산      27
Name: count, dtype: int64
